# Introduction to Astropy: I. Viewing and manipulating FITS images

## Table of Content

I. [Viewing and manipulating FITS images](#I.-Viewing-and-manipulating-FITS-images)     
- I.1 Opening [FITS files, loading the image data, exploring the header](#I.1-Opening-FITS-files,-loading-the-image-data,-exploring-the-header)
- I.2 [Viewing the image and getting basic statistics](#I.2-Viewing-the-image-data-and-getting-basic-statistics)
- I.3 [Displaying the image with log scale](#I.3-Displaying-the-image-with-a-logarithmic-scale)
- I.4 [Basic image math: image stacking](#I.4-Basic-image-math:-image-stacking)
- I.5 [Writing image data to a fits file](#I.5-Writing-image-data-to-a-FITS-file)

XX. [References](#XX-References)


In [ ]:
# As usual, we start with some imports
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

## I. Viewing and manipulating FITS images

Originally, reading and manupulating fits file was included in the package `pyfits` developped by stsci. You may still find that this is the way to do when skimming the internet. This is surely not a bad package to use (most of the early astronomical python development come from stsci ) ... but `astropy` includes that facility as well within a larger growing  astronomical working environment. Hence, we will look here how to do within `astropy.io.fits`. The commands with `pyfits` are essentially the same. 

``astropy.io.fits`` provides a lot of flexibility for reading FITS  files and headers, but most of the time the convenience functions are the easiest way to access the data. 

In [ ]:
# The first step is to import fits
from astropy.io import fits

### I.1 Opening FITS files, loading the image data, exploring the header


``fits.getdata()`` reads just the  data from a FITS file, but with the `header=True` keyword argument will also read the header. 

In [ ]:
image_file = 'HorseHead.fits'
get_data, get_header = fits.getdata(image_file, header=True)

You can also add an 'ext' parameter which specifies the extension of the fits file you are interested in: 
i.e. This can be the extension number:
```
        getdata('in.fits', 0)      # the primary header
        getdata('in.fits', 2)      # the second extension
        getdata('in.fits', ext=2)  # the second extension
```
But the can also be the ``EXTNAME`` value (if unique) (e.g. for HST data):
```
        getdata('in.fits', 'sci')
        getdata('in.fits', extname='sci')  # equivalent
```

For those unfamiliar with FITS headers, they consist of a list of 80 byte `cards`, where a card contains a keyword, a value, and a comment. The keyword and comment must both be strings, whereas the value can be a string or an integer, floating point number, complex number, or True/False. Keywords are usually unique within a header, except in a few special cases.

We can also open the fits and see what it contains. The function `fits.open()` is a higher level function that allows one to perform more advanced operations on a fits file. 
You'll see that the real structure can be a bit more complicated than just a header and a data set in binary format. 

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

The fits is made of "HDU" or "Header Data Units". The function `open()` provides a list of HDU objects.  An HDU (Header Data Unit) is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table. After the above open call, `hdulist[0]` is the primary HDU, `hdulist[1]` is the first extension HDU, etc (if there are any extensions), and so on.

**NOTE:** The open() function will seamlessly open FITS files that have been compressed with gzip, bzip2 or pkzip.

Generally the image information is located in the `PRIMARY` block. The blocks are numbered and can be accessed by indexing `hdu_list`. The scientific data are *not always* in the primary block. Similarly, all the keywords are not stored in the primary HDU. So, remember that if you do not find a keyword of interest in the header (especially while doing `getdata()`), this could simply be because it is not in the pimary header. 

Each element of an HDUList is an HDU object with `.header` and `.data` attributes, which can be used to access the header and data portions of the HDU.

In [ ]:
image_data = hdu_list[0].data

In [ ]:
header_data = hdu_list[0].header

Your data are now stored as a 2-D numpy array.  Want to know the dimensions of the image?  Just look at the `shape` of the array.

In [ ]:
print(type(image_data))
print(image_data.shape)

While your header is saved in a Header object (that has basically the same structure as a dictionary)

In [ ]:
print(type(header_data))

**Note**: You can also access the extension by its extension name (specified in the EXTNAME keyword) instead of its ID in the list, i.e. `hdu_list['Primary'].data`
If there is more than one extension with the same `EXTNAME`, the `EXTVER` value needs to be specified along with the `EXTNAME` as a tuple; e.g.: `scidata = hdulist['sci',2].data`

At this point, we can just close the FITS file.  We have stored everything we wanted to a variable.

In [ ]:
hdu_list.close()

When you close(), the header is still available ... but not always the data. 



**Note:** 

If you don't need to examine the FITS header, you can call `fits.getdata` to bypass the previous steps.

### I.2 Viewing the image data and getting basic statistics

In [ ]:
plt.imshow(image_data, cmap='hot')
plt.colorbar()

# To see more color maps
# http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps
# See also https://jakevdp.github.io/blog/2014/10/16/how-bad-is-your-colormap/

This is nice, but we have to be careful as the x-y scale appearing here is a bit misleading. Indeed, remember that an array is indexed `[row, columns]`, hence the first index corresponds to `y` and the second to `x`. Let's slice the above image:

In [ ]:
f, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].imshow(image_data[:, 0:300], cmap='hot')
ax[1].imshow(image_data[300:600, :], cmap='hot')

The bright star on bottom of the left panel is apparently located at `(170, 738)`, but to get its value you have to look at `image_data[738, 170]`. To avoid doing this mantal change, one sometimes transpose the array before working with it. 

In [ ]:
plt.imshow(image_data, cmap='gray')
plt.plot(170, 738, 'd', color='green')
plt.plot(738, 170, 'x', color='red')
print('value at at the star position (diamond)', image_data[738, 170])
print('value at at the flipped position (cross)', image_data[170, 738])

In summary, to access the values in the array, you have to swap between `x` and `y` positions compared to the x,y you see with `plt.imshow()`, or w.r.t. the position you would measure with another viewer (e.g. `ds9`, `qfitsview`, `aladin`). Note also that the pixel positions given by these viewers start at (1,1) (so you have to subtract 1). WIth those viewers, your image will also appear upside-down compared to what you see with imshow() (unless you work with world coordinates). 

Now let's come back to manipulating our image. 
You can e.g. get some basic statistics about your image 

In [ ]:
print('Min:', np.min(image_data) )
print('Max:', np.max(image_data) )
print('Mean:', np.mean(image_data) )
print('Stdev:', np.std(image_data) )

You can also plot hisograms of some regions of the image. 
To make a histogram with `matplotlib.pyplot.hist()`, you need to cast the data from a 2-D to array to something one dimensional.    

For that purpose, you can use e.g. the iterable python object `img_data.flat`.

In [ ]:
print(type(image_data.flat))
NBINS = 1000
histogram = plt.hist(image_data.flat, NBINS)

### I.3 Displaying the image with a logarithmic scale

Want to use a logarithmic color scale? To do so we need to load the `LogNorm` object from `matplotlib`.

In [ ]:
from matplotlib.colors import LogNorm
plt.imshow(image_data, cmap='gray', norm=LogNorm())

# I chose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000','10,000','20,000'])

### I.4 Basic image math: image stacking

You can perform math with the image data like any other numpy array.  In this particular example, we will stack several images of M13 taken with a ~10'' telescope.

Here, we'll open a series of FITS files and store the data in a list, which will be named `image_concat`.


In [ ]:
image_list = ['M13_blue_000'+n+'.fits' for n in ['1','2','3','4','5'] ]
# The short way  (remember that it is better to use list comprehension)
image_concat = [ fits.getdata(image) for image in image_list]

# The long way
#image_concat = []
#for image in image_list:
#    image_concat.append(fits.getdata(image))

Now I'll stack the images by summing my concatenated list and then visualize the histogram to decide on the best stretch for vizualisation.

In [ ]:
final_image = np.sum(image_concat, axis=0)
image_hist = plt.hist(final_image.flat, 1000)

In [ ]:
plt.imshow(final_image, cmap='gray', vmin=2.e3, vmax=3.e3)
plt.colorbar()

### I.5 Writing image data to a FITS file

This is easy to do with the `writeto()` method.
You will receive an error if the file you are trying to write already exists. To avoid this/allow for overwritting, you should set the option `clobber=True`.

In [ ]:
outfile = 'stacked_M13_blue.fits'

hdu = fits.PrimaryHDU(final_image)
hdu.writeto(outfile, clobber=True)

#### Exercise:

- Select several regions of the image which are apparently sky regions. 
- For simplicity, let's assume that the background is constant over the whole image and subtract the background from the image. 
- Plot the histogram of pixel values for the background subtracted image and display this background subtracted image with adequate cuts.
- Optionally, plot a slice of the frame that crosses the highest density peak of stars in M13.

## Credits:

If you use Astropy directly—or as a dependency to another package—for your work, please remember to include the following acknowledgment at the end of papers:

*This research made use of Astropy, a community-developed core Python package for Astronomy (Astropy Collaboration, 2013).*

Where the astropy paper is 2013, A&A, 558, 33 http://adsabs.harvard.edu//abs/2013A%26A...558A..33A

## XX References

This notebook is mostly based on the astropy tutorials available here: http://www.astropy.org/astropy-tutorials/

- Your reference for using astropy should be the online documentation http://docs.astropy.org/en/latest

- Documentation regarding the wcs module: http://docs.astropy.org/en/stable/visualization/wcsaxes/

- Documentation regarding the use of coordinates: http://docs.astropy.org/en/stable/coordinates/index.html

- The doc of astropy.io.fits also provides relevant information: http://docs.astropy.org/en/stable/io/fits/#f1

- Calabreta and Greisen 2002, A&A 395, 1077, Representations of celestial coordinates in FITS http://adsabs.harvard.edu/abs/2002A%26A...395.1077C

- Regarding `Table` objects and dealing with various i/o within astropy, you should consult those chapters of the doc: http://docs.astropy.org/en/stable/io/unified.html#table-io  and http://docs.astropy.org/en/stable/table/io.html 

- For an in-depth discussion of `Quantity` objects, see the [astropy documentation section](http://docs.astropy.org/en/stable/units/quantity.html). See also http://docs.astropy.org/en/stable/units/ for various informations of interest regarding the use of units in general !

- How bad is your color map ? (aka how not to be fooled by a poor choice of color map): https://jakevdp.github.io/blog/2014/10/16/how-bad-is-your-colormap/

- Github link to astropy tutorial notebooks: https://github.com/astropy/astropy-tutorials/tree/master/tutorials/